---

#Projeto de Disciplina –  IMDb Filmes/ Filmes de 1970 a 2025

---
Universidade Federal de São Carlos

Curso: Bacharelado em Ciência da Computação de Sorocaba

Disciplina: Novas Tecnologias de Bancos de Dados

Professora: Profa. Dra. Sahudy Montenegro González

---

**Grupo 12**

**Integrantes:**

- Gabriel Evangelista Gonçalves da Silva (802791)
- Natalia dos Santos Carvalho (773383)
- Thiago Proença Idro (771064)

---

**Resumo:**

<de 3 a 4 linhas explicando o que o projeto faz>

---

## 1. Introdução

A indústria cinematográfica é um mercado de alto investimento e risco, onde decisões de produção, marketing e distribuição são baseadas em um complexo conjunto de variáveis. Para um gestor de estúdio, produtor ou analista de plataforma de streaming, entender o que define o "sucesso" na indústria de filmes é fundamental. No entanto, o sucesso pode ser medido de várias formas: é o retorno financeiro, a aclamação da crítica especializada, a popularidade com o público ou o reconhecimento da indústria?

Atualmente, as informações necessárias para responder a essas perguntas de forma integrada encontram-se isoladas em diferentes fontes de dados. Por exemplo, dados financeiros detalhados como orçamento e receita podem estar em uma base, enquanto dados de recepção crítica e prêmios estão em outra. Um gestor não consegue, de forma simples, cruzar o impacto do orçamento na nota da crítica ou a relação entre prêmios e a popularidade. Além disso, podemos correlacionar essas informações com informações menos objetivas dos filmes; avaliar essas métricas a partir de atores conhecidos ou gêneros de filme podem trazer uma visão detalhada acima do tema, observando peculiaridades da indústria de filmes.

Este projeto visa prototipar um Data Warehouse (DW) que aborda esse problema de negócio. O objetivo é integrar e consolidar essas múltiplas fontes de dados em um modelo multidimensional coeso. Com o DW implementado, será possível executar análises complexas (OLAP) para identificar padrões e correlações, focando em três eixos principais de análise: a rentabilidade financeira (ROI), a recepção crítica e o desempenho de prêmios e popularidade.



---

## 2. Requisitos do negócio
Para entender o sucesso dos filmes, temos os seguintes requisitos de negócios: 

###Requisito de Negócio 1 (Análise de Rentabilidade): 
Analisar o Retorno sobre Investimento (ROI) e o lucro médio dos filmes, segmentado por gênero e por data de lançamento. O objetivo é identificar quais categorias de filme são historicamente mais rentáveis e quais apresentam maior risco (baixo ROI). **[Natália]**

**Consulta**: Qual o ROI e o lucro médio por gênero e pela data de lançamento?

###Requisito do Negócio 2 (Crítica): 
Identificar a discrepância entre a recepção do público (nota IMDb e rotten tomatoes) e da crítica especializada (nota Metascore). Como essa correlação se distribui por Classificação Indicativa e país de produção? O objetivo é encontrar filmes 'cult' (amados pelo público, odiados pela crítica) ou 'armadilhas de crítico' (o oposto). **[Gabriel]**

**Consulta**: Qual a nota média IMDb, rotten tomatoes e metacritic por classificação indicativa e país de produção?

###Requisito do Negócio 3 (Desempenho): 
Queremos entender o perfil de sucesso dos talentos da indústria. Existe uma correlação entre a popularidade e os prêmios associados a cada pessoa? Ou seja, indivíduos que participam de filmes com alto score de popularidade também tendem a participar de filmes que ganham muitos prêmios? Como essa correlação se apresenta quando filtramos pela função da pessoa? **[Thiago]**

**Consulta**: Qual o score de popularidade e prêmios ganhos e nomeados de pessoas por suas funções?

---

## 3. Fontes dos dados
Para responder aos requisitos de negócio definidos, o projeto utilizará duas APIs como fontes de dados principais. Nenhuma fonte, isoladamente, foi capaz de fornecer todos os atributos necessários para os três pilares de análise. As fontes selecionadas são:

###TMDb API (The Movie Database):

**Domínio Principal:** Dados Financeiros.

**Justificativa:** Esta API é a fonte principal para as métricas de budget (Orçamento) e revenue (Receita), que são a base do Requisito de Negócio 1 (Análise de Rentabilidade). Além disso, contém o campo e popularity com um score de popularidade dos filmes, importante para o Requisito de Negócio 3 (Desempenho).

###OMDb API (Open Movie Database):

**Domínio Principal:** Dados de Recepção Crítica e Reconhecimento.

**Justificativa:** Esta API é a fonte principal para as métricas de imdb e  Metacritic. Além disso, contém o campo textual Awards (Prêmios). Estes atributos são essenciais para o Requisito de Negócio 2 (Crítica) e Requisito de Negócio 3 (Desempenho).

O processo de ETL irá consultar ambas as APIs para cada filme e unificar os resultados.

O cruzamento das informações será feito utilizando o imdb_id como chave de integração, que está presente em ambas as fontes.

Isso permitirá a criação de um modelo multidimensional coeso, onde as métricas financeiras (do TMDB) poderão ser analisadas contra as métricas de crítica (do OMDb) usando as mesmas dimensões (como Gênero, Ano de Lançamento, País, etc.), respondendo assim às consultas complexas que motivaram o projeto.

Os códigos implementados para a extração dos dados está disponível no link: https://github.com/thiagoproenca/IMDB-Filmes 


---

## 4. Camada Bronze

De acordo com os requisitos de negócio projetados, o fato tratado no data warehouse teve como foco métricas de orçamento utilizado, receita adquirida e os diversos tipos de avaliação adquiridas nas fontes de dados; seus atributos coerentes com as métricas observadas nos requisitos de negócio.

O modelo apresenta 3 dimensões: Filme, Gênero e Pessoa. O recurso de tabelas pontes é utilizado para representar a natureza de relações N para N entre filmes e gêneros, filmes e elenco (para os diversos tipos de cargo do elenco). O modelo apresenta também hierarquia nos atributos relacionados a tempo: ano_lancamento, mes_lancamento e dia_lancamento.

### Modelagem

[IMAGEM]

In [0]:
//Trazendo o arquivo parquet com a extração

### Análise Exploratória

aqui dá pra fazer uns sqls super básicos pra analises bem si ples, tipo botar as awards numa tabela

In [0]:
from pyspark.sql.functions import col, sum, when, explode
from pyspark.sql import SparkSession

#Código análise exploratória

from pyspark.sql.functions import col, sum, when
from pyspark.sql import SparkSession

# Iniciar sessão Spark (se ainda não estiver ativa)
spark = SparkSession.builder.appName("ParquetToTable").getOrCreate()

df_parquet = spark.read.parquet("/Volumes/workspace/public/data/extraction_file.parquet")
df_parquet.createOrReplaceTempView("minha_tabela_temp")

resultado = spark.sql("SELECT count(id) FROM minha_tabela_temp")
resultado.show()

+---------+
|count(id)|
+---------+
|    35608|
+---------+



## 5. Camada Prata

###Modelagem ETL

Para a extração de dados, uma mistura entre os dois sites (TMDb e OMDb) pode ser utilizada para alimentar os dados do data warehouse. A partir das API fornecidas, temos a disposição de dados de filme abaixo:

[IMAGEM]

Esse formato ainda dispõe da diferenciação entre tipos de cargo dentro do elenco, diferenciando atores, produtores, escritores e diretores como disposto abaixo:

[IMAGEM]

A partir da extração realizada até o momento de entrega, foi possível definir para a etapa de transformação e limpeza as seguintes operações: 

- Padronização de nome de gênero pós extração para formato coerente com o data warehouse **[Natália]**;
- Transformação no formato das premiações e nomeações de uma única string para dois números independentes **[Gabriel]**;
- Transformação de data de lançamento de uma única string para três números independentes **[Gabriel]**;
- Remoção de filmes sem métrica de avaliação **[Gabriel]**;
- Remoção de filmes sem métrica de orçamento e/ou lucro **[Natália]**;
- Remover filmes com valores em alfabetos diferentes **[Thiago]**;
- Padronizar avaliações em um único formato, de escala 10 ou porcentagem apenas para escala 10 **[Thiago]**.

In [0]:
from pyspark.sql.functions import col, sum, when, explode, split, regexp_extract, coalesce, lit, when
from pyspark.sql import SparkSession
from pyspark.sql.functions import transform
from pyspark.sql.types import IntegerType

In [0]:
spark = SparkSession.builder.appName("ParquetToTable").getOrCreate()

df_parquet = spark.read.parquet("/Volumes/workspace/public/data/extraction_file.parquet")
df_parquet.createOrReplaceTempView("minha_tabela_temp")

resultado = spark.sql("SELECT id, title, release_date, runtime, budget, revenue, genres, credits, awards, ratings  FROM minha_tabela_temp")
resultado.show()

+-------+--------------------+------------+-------+---------+---------+--------------------+--------------------+--------------------+--------------------+
|     id|               title|release_date|runtime|   budget|  revenue|              genres|             credits|              awards|             ratings|
+-------+--------------------+------------+-------+---------+---------+--------------------+--------------------+--------------------+--------------------+
|1084242|          Zootopia 2|  2025-11-26|    107|180000000|233000000|[Animação, Famíli...|{[{Judy Hopps (vo...|1 win & 2 nominat...|{7.8/10, 73/100, ...|
|1419406|  A Sombra Do Perigo|  2025-08-16|    142|        0|174400000|[Ação, Crime, Thr...|{[{Huang Dezhong,...|7 wins & 1 nomina...| {7.5/10, NULL, 80%}|
|1309012|             Altered|  2025-09-18|     85| 15000000|        0|[Ficção científic...|{[{Leon, Tom Felt...|                 N/A|{3.6/10, NULL, NULL}|
|1363123|  Plano em Família 2|  2025-11-11|    106|        0|   

### 1. Padronização de nome de gênero pós extração para formato coerente com o data warehouse [Natália]

Extremamente simples. esse tratamento consiste na separação dos elementos do array pela função explode.

In [0]:
df_generos_separados = df_parquet.withColumn(
    "genre",
    explode(col("genres"))
).select(
    "id",
    "title",
    "runtime",
    "genre"
)
#essa tabela será usada mais adiante para a alimentação da ponte entre generos e filmes

df_generos_separados.createOrReplaceTempView("temp")

resultado = spark.sql("SELECT genre FROM temp GROUP BY genre")
resultado.show()

+-----------------+
|            genre|
+-----------------+
|          Família|
|     Documentário|
|         Thriller|
|         Animação|
|          Romance|
|         Aventura|
|           Guerra|
|             Ação|
|            Crime|
|           Terror|
|         Mistério|
|         História|
|Ficção científica|
|         Fantasia|
|        Cinema TV|
|           Música|
|          Comédia|
|            Drama|
|         Faroeste|
+-----------------+



### 2. Transformação de data de lançamento de uma única string para três números independentes [Gabriel]

explicação bla bla bla

In [0]:
df_temp = df_parquet.withColumn(
    "split_date",
    split(col("release_date"), "-")
)

df_final = df_temp.withColumn(
    "date_array",
    transform(
        col("split_date"),
        lambda x: x.cast(IntegerType())
    )
).drop("split_date")

df_limpo = df_final.drop("release_date")

df_final_desaninhado = df_limpo.select(
    "*",
    col("date_array")[0].alias("ano_lancamento"), 
    col("date_array")[1].alias("mes_lancamento"),
    col("date_array")[2].alias("dia_lancamento")
)

df_post_1 = df_final_desaninhado.select("id", "title", "ano_lancamento", "mes_lancamento", "dia_lancamento", "runtime", "budget", "revenue", "popularity", "original_language", "awards", "vote_count", "ratings")

df_post_1.show()

+-------+--------------------+--------------+--------------+--------------+-------+---------+---------+----------+-----------------+--------------------+----------+--------------------+
|     id|               title|ano_lancamento|mes_lancamento|dia_lancamento|runtime|   budget|  revenue|popularity|original_language|              awards|vote_count|             ratings|
+-------+--------------------+--------------+--------------+--------------+-------+---------+---------+----------+-----------------+--------------------+----------+--------------------+
|1084242|          Zootopia 2|          2025|            11|            26|    107|180000000|233000000|  531.3208|               en|1 win & 2 nominat...|       148|{7.8/10, 73/100, ...|
|1419406|  A Sombra Do Perigo|          2025|             8|            16|    142|        0|174400000|  370.3255|               zh|7 wins & 1 nomina...|       125| {7.5/10, NULL, 80%}|
|1309012|             Altered|          2025|             9|          

### 3. Transformação no formato das premiações e nomeações de uma única string para dois números independentes [Gabriel]

outliers e necessidade de ERs...

In [0]:
resultado = spark.sql("SELECT awards FROM minha_tabela_temp GROUP BY awards")
resultado.display()

awards
13 wins & 7 nominations
Won 1 Oscar. 55 wins & 121 nominations total
1 win & 49 nominations total
38 wins & 30 nominations total
Nominated for 1 Oscar. 21 wins & 12 nominations total
Nominated for 2 Oscars. 15 wins & 45 nominations total
1 win & 12 nominations total
7 wins total
5 wins & 14 nominations total
13 wins & 11 nominations total


In [0]:
df_awards = df_post_1.select(
    "id",
    coalesce(
        when(regexp_extract(col("awards"), r"(\d+)\s+wins?", 1) == "", lit("0")).otherwise(regexp_extract(col("awards"), r"(\d+)\s+wins?", 1)).cast(IntegerType()),
        lit("0").cast(IntegerType())
    ).alias("wins"),
    
    coalesce(
        when(regexp_extract(col("awards"), r"(\d+)\s+nominations?", 1) == "", lit("0")).otherwise(regexp_extract(col("awards"), r"(\d+)\s+nominations?", 1)).cast(IntegerType()),
        lit("0").cast(IntegerType())
    ).alias("nominations")
)

df_post_2 = df_post_1.join(
    df_awards,
    on="id",
    how="left" 
).drop("awards")

df_post_2.show()

### 4. Padronizar avaliações em um único formato, de escala 10 ou porcentagem apenas para escala 10 [Thiago]


inferno de regex. efetivamente tirando símbolos e deixando tudo no padrão 0 a 100 (melhor de tratar do que 0 a 10)

In [ ]:
df_aval = df_post_2.withColumn(
    "imdb_avaliacao",
    col("ratings.imdb")
).withColumn(
    "metacritic_avaliacao",
    col("ratings.metacritic")
).withColumn(
    "rottentomatoes_avaliacao",
    col("ratings.rotten_tomatoes")
).drop("ratings")

def standardize_rating(rating_col, alias_name):
    regex_10_base = r"([\d\.]+)\s*/\s*10"
    regex_100_base = r"(\d+)"

    valor_extraido = coalesce(rating_col, lit("")).cast("string")

    coluna_calculada = (
        when(valor_extraido.rlike(regex_10_base),
             regexp_extract(valor_extraido, regex_10_base, 1).cast("double") * 10
        )
        .when(valor_extraido.rlike(r"(\d+)/100|\d+%"),
              regexp_extract(valor_extraido, regex_100_base, 1).cast("integer")
        )
        .otherwise(lit(0))
    )

    return coluna_calculada.cast(IntegerType()).alias(alias_name)

df_aval_final = df_aval.select(
    "id",
    standardize_rating(col("imdb_avaliacao"), "imdb_avaliacao"),
    when(
        standardize_rating(col("metacritic_avaliacao"), "metacritic_avaliacao") > 100,
        standardize_rating(col("metacritic_avaliacao"), "metacritic_avaliacao").cast("double") / 10
    )
    .otherwise(standardize_rating(col("metacritic_avaliacao"), "metacritic_avaliacao"))
    .cast(IntegerType())
    .alias("metacritic_avaliacao"),
    standardize_rating(col("rottentomatoes_avaliacao"), "rottentomatoes_avaliacao")
)

df_post_3 = df_post_2.join(
    df_aval_final,
    on="id",
    how="left" 
).drop("ratings")

df_post_3.show()

### 5. Remoção de filmes sem métrica de orçamento e/ou lucro [Natália]


ordinário. revenue e budget 0 indicam a falta da informação sobre a métrica.

In [ ]:
df_post_3.createOrReplaceTempView("temp")

df_post_4 = spark.sql("SELECT * FROM temp WHERE revenue > 0 AND budget > 0")
df_post_4.show()

### 6. Remover filmes com valores em alfabetos diferentes [Thiago]

In [ ]:
O regex abaixo descreve o alfabeto latino padrão, e entradas com títulos estrangeiros são removidas

In [ ]:
PADRAO_ALFABETO_COMPLETO = r"^[a-zA-Z0-9\s.,!?'\"()&:\-áÁàÀãÃéÉèÈêÊíÍóÓõÕôÔúÚüÜçÇñÑ]*$"

df_post_5 = df_post_4.filter(
    col("title").rlike(PADRAO_ALFABETO_COMPLETO)
)

df_post_5.show()

### 7. Remoção de filmes sem métrica de avaliação [Gabriel]

super simples. avaliações com 0 indicam a presença de valores null.

In [ ]:
df_post_5.createOrReplaceTempView("temp")

df_post_6 = spark.sql("SELECT * FROM temp WHERE imdb_avaliacao > 0 AND metacritic_avaliacao > 0 AND rottentomatoes_avaliacao > 0")
df_post_6.show()

### 8. Miscelanious e gravações para a camada ouro

Aqui, separamos cada membro do vetor de atores. os dados de df_atores_separados permitem criar as tabelas ponte de ator

In [0]:
df_ator_desaninhado = df_parquet.withColumn(
    "actor_struct",
    explode(col("credits.cast")) 
)

df_atores_separados = df_ator_desaninhado.select(
    col("id"),
    col("title"),
    col("actor_struct.name").alias("actor_name"),
)
#para a ponte entre ator e filme

df_atores_separados.show()

+-------+----------+-----------------+
|     id|     title|       actor_name|
+-------+----------+-----------------+
|1084242|Zootopia 2| Ginnifer Goodwin|
|1084242|Zootopia 2|    Jason Bateman|
|1084242|Zootopia 2|      Ke Huy Quan|
|1084242|Zootopia 2| Fortune Feimster|
|1084242|Zootopia 2|     Andy Samberg|
|1084242|Zootopia 2| David Strathairn|
|1084242|Zootopia 2|       Idris Elba|
|1084242|Zootopia 2|          Shakira|
|1084242|Zootopia 2|Patrick Warburton|
|1084242|Zootopia 2|   Quinta Brunson|
|1084242|Zootopia 2|      Danny Trejo|
|1084242|Zootopia 2|    Nate Torrence|
|1084242|Zootopia 2|      Bonnie Hunt|
|1084242|Zootopia 2|         Don Lake|
|1084242|Zootopia 2|   Michelle Gomez|
|1084242|Zootopia 2|       David Fane|
|1084242|Zootopia 2|       Joe Anoa'i|
|1084242|Zootopia 2|      Phil Brooks|
|1084242|Zootopia 2|Stephanie Beatriz|
|1084242|Zootopia 2|Wilmer Valderrama|
+-------+----------+-----------------+
only showing top 20 rows


Abaixo, o desaninhar dos membros do elenco. os dados de df_cast_separado permitem criar as tabelas ponte de produtores, escritores e diretores.

In [0]:
df_cast_desaninhado = df_parquet.withColumn(
    "crew_struct",
    explode(col("credits.crew")) 
)

df_cast_separado = df_cast_desaninhado.filter("crew_struct.department == 'Directing' OR crew_struct.department == 'Production' OR crew_struct.department == 'Writing'").select(
    col("id"),
    col("crew_struct.name"),
    col("crew_struct.department").alias("department"),
)
#para as pontes entre filme e prod, writ, direct

df_cast_separado.show()

+-------+--------------------+----------+
|     id|                name|department|
+-------+--------------------+----------+
|1084242|          Jared Bush| Directing|
|1084242|          Jared Bush|   Writing|
|1084242|        Yvett Merino|Production|
|1084242|        Byron Howard| Directing|
|1084242|          Jared Bush|Production|
|1084242|        Jennifer Lee|Production|
|1084242|        Grace C. Kim|Production|
|1084242|         Carrie Liao|   Writing|
|1084242|      David VanTuyle|   Writing|
|1084242|           Ariana Oh|   Writing|
|1084242|     Kennedy Tarrell|   Writing|
|1084242|         Nancy Kruse|   Writing|
|1084242|       Jeremy Spears|   Writing|
|1084242|      Hikari Toriumi|   Writing|
|1084242|          Mai Shirai|   Writing|
|1084242|          Ryan Green|   Writing|
|1084242|          Kyu Ri Ahn|   Writing|
|1084242|     Miguel Baltazar|   Writing|
|1084242|Alberto Rodriguez...|   Writing|
|1084242|       Tom Caulfield|   Writing|
+-------+--------------------+----

In [0]:
df_cast_separado.createOrReplaceTempView("temp")

resultado = spark.sql("SELECT department FROM temp GROUP BY department")
resultado.show()

+----------+
|department|
+----------+
| Directing|
|Production|
|   Writing|
+----------+



Por fim, eliminação de outliers de métrica com base em pesquisas reais sobre dados de receita adquirida e orçamento gasto. Únicos outliers importantes para retirar do DB

In [ ]:
df_post_6.createOrReplaceTempView("temp")

df_post_7 = spark.sql("SELECT * FROM temp WHERE revenue < 3000000000 AND budget < 447000000")

df_post_7.show()

## 6. Camada Ouro

###Dimensões

**Dimensão Genêro**

In [0]:
from pyspark.sql.functions import monotonically_increasing_id, col

df_dim_genero = df_generos_separados.select(col("genre").alias("nome_genero")).distinct() \
    .withColumn("id_genero", monotonically_increasing_id())

df_dim_genero.createOrReplaceTempView("Dim_Genero")
print("Dimensão Gênero criada.")

display(df_dim_genero)

Dimensão Gênero criada.


nome_genero,id_genero
Família,0
Documentário,1
Thriller,2
Animação,3
Romance,4
Aventura,5
Guerra,6
Ação,7
Crime,8
Terror,9


**Dimensão Pessoa**

In [0]:
df_todos_nomes = df_atores_separados.select(col("actor_name").alias("nome_pessoa")) \
    .union(df_cast_separado.select(col("name").alias("nome_pessoa")))

df_dim_pessoa = df_todos_nomes.distinct() \
    .withColumn("id_pessoa", monotonically_increasing_id())

df_dim_pessoa.createOrReplaceTempView("Dim_Pessoa")
print("Dimensão Pessoa criada.")

display(df_dim_pessoa)

Dimensão Pessoa criada.


nome_pessoa,id_pessoa
Danielle Brooks,0
Chad Michael Murray,1
Olga Dzyurak,2
Anna Camp,3
Sanjay Hari,4
Jim Gaffigan,5
Isac Creato Gomes,6
Matthieu Carle,7
Yuta Uemura,8
Eric Daniel,9


**Dimensão Filme**

In [0]:
print(df_post_6.columns)


['id', 'title', 'ano_lancamento', 'mes_lancamento', 'dia_lancamento', 'runtime', 'budget', 'revenue', 'popularity', 'original_language', 'vote_count', 'wins', 'nominations', 'imdb_avaliacao', 'metacritic_avaliacao', 'rottentomatoes_avaliacao']


In [0]:
df_dim_filme = df_post_6.select(
    col("id").alias("id_filme"), 
    "title", 
    col("ano_lancamento").alias("ano"), 
    "runtime"
).distinct()

df_dim_filme.createOrReplaceTempView("Dim_Filme")
print("Dimensão Filme criada.")

display(df_dim_filme)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-6284332321138042>, line 8
      1 df_dim_filme = df_post_6.select(
      2     col("id").alias("id_filme"), 
      3     "title", 
      4     col("ano_lancamento").alias("ano"), 
      5     "runtime"
      6 ).distinct()
----> 8 df_dim_filme.createOrReplaceTempView("Dim_Filme")
      9 print("Dimensão Filme criada.")
     11 display(df_dim_filme)

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:2075, in DataFrame.createOrReplaceTempView(self, name)
   2071 def createOrReplaceTempView(self, name: str) -> None:
   2072     command = plan.CreateView(
   2073         child=self._plan, name=name, is_global=False, replace=True
   2074     ).command(session=self._session.client)
-> 2075     _, _, ei = self._session.client.execute_command(command, self._plan.observations)
   2076     self._e

###Pontes

**Ponte Filme-Gênero**

In [0]:
df_ponte_genero = df_generos_separados.select(col("id").alias("id_filme"), col("genre").alias("nome_genero")) \
    .join(df_dim_genero, "nome_genero") \
    .select("id_filme", "id_genero").distinct()

df_ponte_genero.createOrReplaceTempView("Ponte_Filme_Genero")
print("Ponte Gênero criada.")

**Ponte Filme-Ator**

In [0]:
df_ponte_ator = df_atores_separados.select(col("id").alias("id_filme"), col("actor_name").alias("nome_pessoa")) \
    .join(df_dim_pessoa, "nome_pessoa") \
    .select("id_filme", "id_pessoa").distinct()

df_ponte_ator.createOrReplaceTempView("Ponte_Filme_Ator")
print("Ponte Ator criada.")

**Ponte Filme-Diretor**

In [0]:
df_ponte_diretor = df_cast_separado.filter(col("department") == "Directing") \
    .select(col("id").alias("id_filme"), col("name").alias("nome_pessoa")) \
    .join(df_dim_pessoa, "nome_pessoa") \
    .select("id_filme", "id_pessoa").distinct()

df_ponte_diretor.createOrReplaceTempView("Ponte_Filme_Diretor")
print("Ponte Diretor criada.")

**Ponte Filme-Escritor**

In [0]:
df_ponte_escritor = df_cast_separado.filter(col("department") == "Writing") \
    .select(col("id").alias("id_filme"), col("name").alias("nome_pessoa")) \
    .join(df_dim_pessoa, "nome_pessoa") \
    .select("id_filme", "id_pessoa").distinct()

df_ponte_escritor.createOrReplaceTempView("Ponte_Filme_Escritor")
print("Ponte Escritor criada.")

**Ponte Filme-Produtor**

In [0]:
df_ponte_produtor = df_cast_separado.filter(col("department") == "Production") \
    .select(col("id").alias("id_filme"), col("name").alias("nome_pessoa")) \
    .join(df_dim_pessoa, "nome_pessoa") \
    .select("id_filme", "id_pessoa").distinct()

df_ponte_produtor.createOrReplaceTempView("Ponte_Filme_Produtor")
print("Ponte Produtor criada.")

**Tabela Fato**

In [0]:
from pyspark.sql.functions import regexp_extract, lit 
from pyspark.sql.types import FloatType, IntegerType

df_fato = df_post_6.select(
    col("id").alias("id_filme"),
    col("budget").alias("orcamento"),
    col("revenue").alias("receita"),
    col("popularity").alias("popularidade"),
    col("imdb_avaliacao"),
    col("metacritic_avaliacao"),
    col("wins").alias("premios_vencidos")
)

df_fato.createOrReplaceTempView("Fato_Metricas")
print("Tabela Fato criada com sucesso!")

In [0]:
%sql
SELECT 
    g.nome_genero AS Genero,
    f.ano AS Ano,
    ROUND(AVG(ft.receita - ft.orcamento), 2) AS Lucro_Medio,
    -- ROI % (Evita divisão por zero)
    ROUND(AVG(((ft.receita - ft.orcamento) / NULLIF(ft.orcamento, 0)) * 100), 2) AS ROI_Percentual
FROM Fato_Metricas ft
JOIN Dim_Filme f ON ft.id_filme = f.id_filme
JOIN Ponte_Filme_Genero pg ON f.id_filme = pg.id_filme
JOIN Dim_Genero g ON pg.id_genero = g.id_genero
GROUP BY g.nome_genero, f.ano
ORDER BY Lucro_Medio DESC

In [0]:
%sql
SELECT 
    g.nome_genero AS Genero,
    COUNT(ft.id_filme) AS Qtd_Filmes,
    ROUND(AVG(ft.imdb_avaliacao), 1) AS Media_IMDb,
    ROUND(AVG(ft.metacritic_avaliacao), 1) AS Media_Metascore,
    -- Positivo = Público gostou mais. Negativo = Crítica gostou mais.
    ROUND(AVG(ft.imdb_avaliacao - ft.metacritic_avaliacao), 1) AS Discrepancia
FROM Fato_Metricas ft
JOIN Dim_Filme f ON ft.id_filme = f.id_filme
JOIN Ponte_Filme_Genero pg ON f.id_filme = pg.id_filme
JOIN Dim_Genero g ON pg.id_genero = g.id_genero
GROUP BY g.nome_genero
HAVING COUNT(ft.id_filme) > 10
ORDER BY Discrepancia DESC;

In [0]:
%sql
SELECT pes.nome_pessoa AS Nome, 'Ator' AS Funcao, 
       ROUND(AVG(ft.popularidade), 2) AS Popularidade_Media, 
       SUM(ft.premios_vencidos) AS Total_Premios
FROM Dim_Pessoa pes
JOIN Ponte_Filme_Ator ponte ON pes.id_pessoa = ponte.id_pessoa
JOIN Dim_Filme f ON ponte.id_filme = f.id_filme
JOIN Fato_Metricas ft ON f.id_filme = ft.id_filme
GROUP BY pes.nome_pessoa

UNION ALL

SELECT pes.nome_pessoa AS Nome, 'Diretor' AS Funcao, 
       ROUND(AVG(ft.popularidade), 2) AS Popularidade_Media, 
       SUM(ft.premios_vencidos) AS Total_Premios
FROM Dim_Pessoa pes
JOIN Ponte_Filme_Diretor ponte ON pes.id_pessoa = ponte.id_pessoa
JOIN Dim_Filme f ON ponte.id_filme = f.id_filme
JOIN Fato_Metricas ft ON f.id_filme = ft.id_filme
GROUP BY pes.nome_pessoa

UNION ALL

SELECT pes.nome_pessoa AS Nome, 'Escritor' AS Funcao, 
       ROUND(AVG(ft.popularidade), 2) AS Popularidade_Media, 
       SUM(ft.premios_vencidos) AS Total_Premios
FROM Dim_Pessoa pes
JOIN Ponte_Filme_Escritor ponte ON pes.id_pessoa = ponte.id_pessoa
JOIN Dim_Filme f ON ponte.id_filme = f.id_filme
JOIN Fato_Metricas ft ON f.id_filme = ft.id_filme
GROUP BY pes.nome_pessoa

UNION ALL

SELECT pes.nome_pessoa AS Nome, 'Produtor' AS Funcao, 
       ROUND(AVG(ft.popularidade), 2) AS Popularidade_Media, 
       SUM(ft.premios_vencidos) AS Total_Premios
FROM Dim_Pessoa pes
JOIN Ponte_Filme_Produtor ponte ON pes.id_pessoa = ponte.id_pessoa
JOIN Dim_Filme f ON ponte.id_filme = f.id_filme
JOIN Fato_Metricas ft ON f.id_filme = ft.id_filme
GROUP BY pes.nome_pessoa

ORDER BY Total_Premios DESC

## 7. Bibliografia
	
IMDb. IMDb: Ratings, Reviews, and Where to Watch the Best Movies and TV Shows. Disponível em: https://www.imdb.com/pt/ (acesso em 04/11/2025).

TMDB. The Movie Database (TMDB). Disponível em: https://www.themoviedb.org/ (acesso em 04/11/2025).

OMDb API. The Open Movie Database (OMDb). Disponível em: https://www.omdbapi.com/ (acesso em 04/11/2025).

Metacritic. Movie Reviews, TV Reviews, Game Reviews, and Music Reviews - Metacritic. Disponível em: https://www.metacritic.com/ (acesso em 04/11/2025).

Rotten Tomatoes. Rotten Tomatoes: Movies | TV Shows | Movie Trailers | Reviews. Disponível em: https://www.rottentomatoes.com/ (acesso em 04/11/2025).